In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset
data_path = "datasetIRB2400.csv"  # Update this path if different
data = pd.read_csv(data_path)

In [2]:
# Inspect the data
print(data.head())
print(data.info())

# Drop unnecessary columns or rows if any (customize as needed)
# For instance, if there are null values:
data = data.dropna()

         x         y          z     yaw   pitch    roll   q1_in   q2_in  \
0 -80.9148 -121.7499  1896.4867 -0.6941  0.9229 -1.4411 -2.2407 -0.7975   
1 -80.9148 -121.7499  1896.4867 -0.6941  0.9229 -1.4411 -2.2233 -0.8511   
2 -80.9148 -121.7499  1896.4867 -0.6941  0.9229 -1.4411 -2.2567 -0.9468   
3 -80.9148 -121.7499  1896.4867 -0.6941  0.9229 -1.4411 -2.1557 -0.8205   
4 -80.9148 -121.7499  1896.4867 -0.6941  0.9229 -1.4411 -2.3267 -0.9279   

    q3_in   q4_in   q5_in   q6_in  q1_out  q2_out  q3_out  q4_out  q5_out  \
0  0.6551 -3.2989  1.1212  5.7052 -2.2347 -0.8949  0.6113 -3.2841  1.2117   
1  0.5966 -3.2320  1.1662  5.6429 -2.2347 -0.8949  0.6113 -3.2841  1.2117   
2  0.5820 -3.3165  1.1220  5.5944 -2.2347 -0.8949  0.6113 -3.2841  1.2117   
3  0.5532 -3.1993  1.2697  5.7474 -2.2347 -0.8949  0.6113 -3.2841  1.2117   
4  0.6722 -3.2867  1.2266  5.6304 -2.2347 -0.8949  0.6113 -3.2841  1.2117   

   q6_out  
0  5.6792  
1  5.6792  
2  5.6792  
3  5.6792  
4  5.6792  
<class 'pandas

In [3]:
# Update the feature and target column names based on your dataset
# "q1_in","q2_in","q3_in","q4_in","q5_in", "q6_in","x", "y", "z"
X = data.drop(columns=["q1_out","q2_out","q3_out","q4_out","q5_out", "q6_out"])
y = data[["q1_out","q2_out","q3_out","q4_out","q5_out", "q6_out"]]

In [4]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save the processed data if needed
pd.DataFrame(X_train_scaled).to_csv("./X_train_scaled.csv", index=False)
pd.DataFrame(X_test_scaled).to_csv("./X_test_scaled.csv", index=False)
pd.DataFrame(y_train).to_csv("./y_train.csv", index=False)
pd.DataFrame(y_test).to_csv("./y_test.csv", index=False)

print("Preprocessing complete. Data is ready for training.")


Preprocessing complete. Data is ready for training.


In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

In [10]:
# Build the model
model = Sequential([
    Dense(256, input_dim=X_train.shape[1], activation='relu'),  # Input layer
    BatchNormalization(),
    Dropout(0.3),

    Dense(512, activation='relu'),  # Hidden layer 1
    BatchNormalization(),
    Dropout(0.4),

    Dense(512, activation='relu'),  # Hidden layer 2
    BatchNormalization(),
    Dropout(0.4),

    Dense(256, activation='relu'),  # Hidden layer 3
    BatchNormalization(),
    Dropout(0.3),

    Dense(128, activation='relu'),  # Hidden layer 4
    BatchNormalization(),
    Dropout(0.3),

    Dense(64, activation='relu'),  # Hidden layer 5
    BatchNormalization(),
    Dropout(0.2),

    Dense(y_train.shape[1], activation='linear')  # Output layer
])

# Create custom metrics using Keras metric classes
class CoordinateMSE(tf.keras.metrics.Metric):
    def __init__(self, coord_idx, name='coordinate_mse', **kwargs):
        super().__init__(name=name, **kwargs)
        self.coord_idx = coord_idx
        self.mse = self.add_weight(name='mse', initializer='zeros')
        self.count = self.add_weight(name='count', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        error = tf.square(y_true[:, self.coord_idx] - y_pred[:, self.coord_idx])
        self.mse.assign_add(tf.reduce_sum(error))
        self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.mse / self.count

    def reset_state(self):
        self.mse.assign(0.)
        self.count.assign(0.)

# Compile the model
optimizer = Adam(learning_rate=1e-3)
early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
model.compile(
    optimizer=optimizer,
    loss='mse',
    metrics=[
        'mse',
        tf.keras.metrics.RootMeanSquaredError(name='rmse'),
        CoordinateMSE(coord_idx=0, name='x_mse'),
        CoordinateMSE(coord_idx=1, name='y_mse'),
        CoordinateMSE(coord_idx=2, name='z_mse')
    ]
)

c:\Users\umair\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
# Train the model with corrected custom metrics functions
def x_mse(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true[:, 0] - y_pred[:, 0]))

def y_mse(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true[:, 1] - y_pred[:, 1]))

def z_mse(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true[:, 2] - y_pred[:, 2]))

# Train the model
history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=5,
    batch_size=64,
    callbacks=[early_stop],
    verbose=1
)

# Evaluate the model
results = model.evaluate(X_test_scaled, y_test, verbose=1)

# Print all metrics
print(f"Test Loss (MSE): {results[1]}")
print(f"Test RMSE: {results[2]}")
print(f"X coordinate MSE: {results[3]}")
print(f"Y coordinate MSE: {results[4]}")
print(f"Z coordinate MSE: {results[5]}")

# Optional: Calculate and print the RMSE for each coordinate
print(f"X coordinate RMSE: {np.sqrt(results[3])}")
print(f"Y coordinate RMSE: {np.sqrt(results[4])}")
print(f"Z coordinate RMSE: {np.sqrt(results[5])}")

# Save the trained model
model.save("robot_kinematics_model.h5")

Epoch 1/5
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 25s 8ms/step - loss: 0.2754 - mse: 0.2754 - rmse: 0.5248 - x_mse: 0.2529 - y_mse: 0.1493 - z_mse: 0.0931 - val_loss: 0.0237 - val_mse: 0.0237 - val_rmse: 0.1538 - val_x_mse: 0.0217 - val_y_mse: 0.0230 - val_z_mse: 0.0195
Epoch 2/5
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 24s 8ms/step - loss: 0.2633 - mse: 0.2633 - rmse: 0.5132 - x_mse: 0.2455 - y_mse: 0.1441 - z_mse: 0.0845 - val_loss: 0.0215 - val_mse: 0.0215 - val_rmse: 0.1467 - val_x_mse: 0.0184 - val_y_mse: 0.0180 - val_z_mse: 0.0159
Epoch 3/5
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 23s 8ms/step - loss: 0.2552 - mse: 0.2552 - rmse: 0.5051 - x_mse: 0.2372 - y_mse: 0.1380 - z_mse: 0.0783 - val_loss: 0.0215 - val_mse: 0.0215 - val_rmse: 0.1466 - val_x_mse: 0.0172 - val_y_mse: 0.0195 - val_z_mse: 0.0139
Epoch 4/5
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 24s 8ms/step - loss: 0.2509 - mse: 0.2509 - rmse: 0.5009 - x_mse: 0.2322 - y_mse: 0.1316 - z_mse: 0.0751 - val_loss: 0.0207 - val_mse: 0.0207 - val_rmse: 0.1440 - val_x_mse:

Test Loss (MSE): 0.020147420465946198
Test RMSE: 0.1419416069984436
X coordinate MSE: 0.019628271460533142
Y coordinate MSE: 0.014301941730082035
Z coordinate MSE: 0.014088207855820656
X coordinate RMSE: 0.14010093311799585
Y coordinate RMSE: 0.11959072593676331
Z coordinate RMSE: 0.11869375660000257


In [10]:
import numpy as np

# Example single input (replace with your actual input row index or values)
single_input = np.array(X_train.iloc[0])  # Extract the row and convert to a NumPy array

# Reshape the input to ensure it's 2D (one sample with multiple features)
single_input = single_input.reshape(1, -1)  # Now it has shape (1, n_features)

# Scale the input using the same scaler as during training
single_input_scaled = scaler.transform(single_input)  # Assuming you used StandardScaler

# Predict the output using the trained model
predicted_output = model.predict(single_input_scaled)

print("Predicted output:")
print(predicted_output)

c:\Users\umair\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step
Predicted output:
[[-0.3589563  1.0244242 -0.8013429 -2.9221222  1.2968899  1.7713171]]


In [11]:
import numpy as np
import pandas as pd

# Generate 20 random indices within the range of the dataset (0 to 299999)
random_indices = np.random.randint(0, 200000, size=2)

# random_indices = 23311

# Select 20 random samples from the dataset
random_samples = X_train.iloc[random_indices]  # Features
random_actual_outputs = y_train.iloc[random_indices]  # Actual target outputs

# Scale the inputs using the same scaler as during training
random_samples_scaled = scaler.transform(random_samples)

# Predict the outputs using the trained model
predicted_outputs = model.predict(random_samples_scaled)

print("Random 20 Samples:")
print(random_samples)
print("\nActual Outputs:")
print(random_actual_outputs)
print("\nPredicted Outputs:")
print(predicted_outputs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
Random 20 Samples:
               x         y          z     yaw   pitch    roll   q1_in   q2_in  \
92790  -174.5724 -509.1983  1487.1044 -1.4991  0.7108 -0.9685  1.1577 -1.2782   
117344  631.0593 -389.7585   918.7209 -2.0306  0.4093  0.2231 -0.4129  0.7726   

         q3_in   q4_in   q5_in   q6_in  
92790  -0.1382  2.0153 -1.5716  1.0614  
117344 -0.3523 -2.5262  0.5990 -5.2470  

Actual Outputs:
        q1_out  q2_out  q3_out  q4_out  q5_out  q6_out
92790   1.0985 -1.2891 -0.0703  2.0297 -1.5885  1.0505
117344 -0.5076  0.7601 -0.4265 -2.4533  0.6776 -5.3273

Predicted Outputs:
[[ 1.4039541  -1.2455312  -0.1707567   2.1034045  -1.4951164   1.0409797 ]
 [-0.17429204  0.8754224  -0.3326313  -2.5037773   0.7495024  -5.0533857 ]]


In [12]:
import numpy as np
import pandas as pd

# Generate 20 random indices within the range of the dataset (0 to 299999)
random_indices = np.random.randint(0, 200000, size=2)

# Select 20 random samples from the dataset
random_samples = X_train.iloc[random_indices]  # Features
random_actual_outputs = y_train.iloc[random_indices]  # Actual target outputs

# Scale the inputs using the same scaler as during training
random_samples_scaled = scaler.transform(random_samples)

# Predict the outputs using the trained model
predicted_outputs = model.predict(random_samples_scaled)

print("Random 20 Samples:")
print(random_samples)
print("\nActual Outputs:")
print(random_actual_outputs)
print("\nPredicted Outputs:")
print(predicted_outputs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Random 20 Samples:
               x         y          z     yaw   pitch    roll   q1_in   q2_in  \
212327 -686.8876 -346.4181  1352.3453  1.0975  1.3345 -2.3418  0.5545 -1.4165   
30132   446.3427 -145.9601  1116.7101 -1.3425 -1.1071  0.7696 -0.2953  0.4892   

         q3_in   q4_in   q5_in   q6_in  
212327  0.0642 -0.7056 -0.2994 -2.5594  
30132  -0.5661 -0.2213 -1.1785  5.9708  

Actual Outputs:
        q1_out  q2_out  q3_out  q4_out  q5_out  q6_out
212327  0.4894 -1.4765 -0.0289 -0.7286 -0.3076 -2.5738
30132  -0.3454  0.3989 -0.5330 -0.1816 -1.1117  5.9738

Predicted Outputs:
[[ 0.641224   -1.4181796   0.0408111  -0.55604404 -0.22010803 -2.5059502 ]
 [-0.21646768  0.44913214 -0.5916454  -0.04699313 -1.1392686   5.636353  ]]
